In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs 
from matplotlib import ticker, cm

In [ ]:
accidents_uned = pd.read_csv(".\Accidents_June20.csv")

In [ ]:
# source: https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-rev11/data-download
# NASA: socioeconomic data and applications center (SEDAC)
pop = np.genfromtxt(".\popdensity3\gpw_v4_population_density_rev11_2020_2pt5_min.asc", skip_header=6)

# betere resolutie (maar niet genoeg memory op pc)
#pop = np.genfromtxt(".\popdensity\gpw_v4_population_density_rev11_2020_30_sec_1.asc", skip_header=6)

In [ ]:
lons = np.arange(-180,180,360/np.shape(pop)[1]) + 360/np.shape(pop)[1]/2
lats = np.arange(90,-90,-180/np.shape(pop)[0]) + 180/np.shape(pop)[0]/2

In [ ]:
masked_pop = np.ma.masked_where(pop == -9999,pop)

### Maak een plaatje

In [ ]:
fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

#ax.stock_img()
ax.coastlines()
plt.xticks(np.arange(-180,181,60), fontsize=15)  
plt.yticks(np.arange(-90,91,30), fontsize=15) 
CS = plt.contourf(lons,lats,masked_pop+0.01,cmap=plt.cm.jet, locator=ticker.LogLocator(), levels=np.logspace(-3,5,33))
for c in CS.collections:
    c.set_edgecolor("face")
plt.xlim(-180,180)
#plt.ylim(20,60)
plt.xlabel('longitude [$\degree$E]', fontsize=15)
plt.ylabel('latitude [$\degree$N]', fontsize=15)
plt.title('Population density [persons/km$^2$]', fontsize=15)
plt.grid(True)
plt.colorbar(fraction=0.040, pad=0.04)

# January
plt.show()

### Maak de nieuwe database, met population density per accident

In [ ]:
popdens_vals = np.zeros(np.size(accidents_uned["Start_Lat"]))
#itts = 0

for i,x in enumerate(accidents_uned["Start_Lat"]):
    n_lat = np.argmin(np.abs(lats-accidents_uned["Start_Lat"][i]))
    n_lon = np.argmin(np.abs(lons-accidents_uned["Start_Lng"][i]))
    popdens_vals[i] = masked_pop.data[n_lat][n_lon]
    #itts += 1
    #if itts > 50:
        #break

In [ ]:
accidents_uned["Population_Density"] = popdens_vals

In [ ]:
#accidents_uned.to_csv("./Accidents_Popdensity.csv", chunksize=1000)